## My Snoring 

Use the data of the sound snoring sensor and visualise it here. 

- Use snoring sensor on RPi and store it on the InfluxDB
- Use Grafana (hass.io) to get the data from the device.
- Download CSV from Grafana and import it here.

## Contents
0. Get the data and load into Pandas df
1. Data inspection, cleaning and visualisation
2. Statistical analysis
3. Store the results as json

## 0. Get the data and load into Pandas df

In [82]:
pwd

'C:\\Users\\31653\\Documents\\GitHub\\Notebooks'

In [1]:
import glob
my_csvs = glob.glob('*.csv')
my_csvs

['2021-12-5_Features_.csv',
 'foo.csv',
 'OEtest_210505_youtube_traffic.csv',
 'OEtest_210506_youtubetraffic.csv',
 'Snoring-data-2021-12-03 09_15_24.csv',
 'Snoring-data-2021-12-04 11_41_48.csv',
 'Snoring-data-2021-12-04 11_51_51.csv',
 'Snoring-data-2021-12-06 19_03_36.csv',
 'Snoring-data-2021-12-10 13_27_41.csv',
 'snoring_2021_12_3-4b.csv',
 'UMTMVS.csv']

In [2]:
import pandas as pd
snoring_df=pd.read_csv('Snoring-data-2021-12-10 13_27_41.csv')
snoring_df.tail()

,Time,my_snoring.mean
32996,2021-12-10 07:49:56,0.1300
32997,2021-12-10 07:49:57,0.1640
32998,2021-12-10 07:49:58,0.1050
32999,2021-12-10 07:49:59,0.0667
33000,2021-12-10 07:50:00,NaN


## 1. Data inspection, cleaning and visualisation


In [3]:
#show colums
snoring_df.columns

Index(['Time', 'my_snoring.mean'], dtype='object')

In [4]:
#describe data
snoring_df.describe()

,my_snoring.mean
count,32996.000000
mean,0.085032
std,0.092839
min,0.037200
25%,0.064900
50%,0.065800
75%,0.066300
max,1.000000


In [ ]:
#remove NaN

In [26]:
#visualise the data. To do: larger plot and mean.values
snoring_df.plot.scatter(x="Time", y="my_snoring.mean")

<AxesSubplot:xlabel='Time', ylabel='my_snoring.mean'>

Error in callback <function flush_figures at 0x0000025318FDD3A0> (for post_execute):


KeyboardInterrupt: 

## 2. Statistical analysis

In [5]:
print(snoring_df.sort_values('my_snoring.mean', ascending='False'))

                      Time  my_snoring.mean
23608  2021-12-10 05:13:28           0.0372
3752   2021-12-09 23:42:32           0.0440
11527  2021-12-10 01:52:07           0.0450
3414   2021-12-09 23:36:54           0.0511
23609  2021-12-10 05:13:29           0.0527
...                    ...              ...
3642   2021-12-09 23:40:42              NaN
14028  2021-12-10 02:33:48              NaN
14029  2021-12-10 02:33:49              NaN
14030  2021-12-10 02:33:50              NaN
33000  2021-12-10 07:50:00              NaN

[33001 rows x 2 columns]


In [6]:
# add a column 'bins' to the df and create bins with labels

snoring_df['bins'] = pd.cut(snoring_df['my_snoring.mean'], bins=[0.0, 0.10, 0.50, 1.00],
                    labels=['no snoring', 'light snoring', 'loud snoring']) 


In [7]:
snoring_df.tail()

,Time,my_snoring.mean,bins
32996,2021-12-10 07:49:56,0.1300,light snoring
32997,2021-12-10 07:49:57,0.1640,light snoring
32998,2021-12-10 07:49:58,0.1050,light snoring
32999,2021-12-10 07:49:59,0.0667,no snoring
33000,2021-12-10 07:50:00,NaN,NaN


In [8]:
total_length = snoring_df['bins'].count()
total_length

32996

In [11]:
stats_df = pd.DataFrame()
stats_df['class'] =snoring_df['bins'].value_counts()
stats_df

,class
no snoring,30223
light snoring,2347
loud snoring,426


In [23]:
no_snoring =stats_df['class'][0]
light_snoring=stats_df['class'][1]
loud_snoring=stats_df['class'][2]

In [24]:
time_snoring = round(((light_snoring+loud_snoring)/60),2)
print("you've snored: " +str(time_snoring) + " minutes")

you've snored: 46.22 minutes


In [25]:
percentage = ((loud_snoring+light_snoring) / total_length)*100
print ("that is " + str(round(percentage,2)) + "% the measured time")

that is 8.4% the measured time


In [31]:
loud_perc = round(loud_snoring/total_length*100,2)
print ("loud snoring was " + str(loud_perc) + "% of the measured time")

loud snoring was 1.29% of the measured time


### 2b. First and last time of snoring

## 3. Store the results